In [2]:
import pandas as pd
import re 
import numpy as np
jeopardy=pd.read_csv("JEOPARDY_CSV.csv")
jeopardy.columns=jeopardy.columns.str.strip()


/Users/Jaspinder/anaconda/lib/python3.6/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory: 'sysctl'
  stacklevel=stacklevel + 1):


In [3]:
##Normalize the text in QUESTIONS and ANSWERS columns 

def normalize_text(text):
    text=text.lower()
    text=re.sub('[^a-zA-Z0-9\s]','',text)
    return text
jeopardy['clean_question']=jeopardy['Question'].apply(normalize_text)
jeopardy['clean_answer']=jeopardy['Answer'].apply(normalize_text)
    
jeopardy.head()
    

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams


In [4]:
###Normalize the values in Value columns
def normalize_value(value):
    value=re.sub('[^a-zA-Z0-9]','',value)
    try:
        value = int(value)
    except Exception:
        value = 0
    return value
jeopardy['clean_value']=jeopardy['Value'].apply(normalize_value)
jeopardy['Air Date']=pd.to_datetime(jeopardy['Air Date'])
print(jeopardy.dtypes)
jeopardy.head()


Show Number                int64
Air Date          datetime64[ns]
Round                     object
Category                  object
Value                     object
Question                  object
Answer                    object
clean_question            object
clean_answer              object
clean_value                int64
dtype: object


,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe,200
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona,200
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds,200
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams,200


In [5]:
###How often the answer is deducible from the question
def deductible(row):
    split_answer=row['clean_answer'].split()
    if 'the' in split_answer:
        split_answer.remove('the')
    split_question=row['clean_question'].split()
    if 'the' in split_question:
        split_question.remove('the')
    if len(split_answer)==0:
        return 0
    match_count=0
    for i in split_answer:
        if i in split_question:
            match_count=match_count+1
    answer_in_question=match_count/len(split_answer)
    return answer_in_question
jeopardy['answer_in_question']=jeopardy.apply(deductible,axis=1)
jeopardy.head()



,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value,answer_in_question
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200,0.0
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe,200,0.0
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona,200,0.0
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds,200,0.0
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams,200,0.0


In [6]:
mean=np.mean(jeopardy['answer_in_question'])
mean
jeopardy['count_fff']=1


In [7]:
s=jeopardy.pivot_table(index='clean_question',values=['count_fff'],aggfunc='count')
s.sort_values(['count_fff'],ascending=False)
s=s[s['count_fff']>1]
x=s['count_fff'].sum()


In [8]:
###Rough Work
y=jeopardy['count_fff'].sum()

In [9]:
###Rough Work 

1918/216930

0.008841561794127139

In [22]:
question_overlap=[]
terms_used=set()
for i,row in jeopardy.iterrows():
    split_question=row['clean_question'].split()
    split_question = [q for q in split_question if len(q) > 5]
    match_count=0
    for word in split_question:
        if word in terms_used:
            match_count=match_count+1
        else:
            terms_used.add(word)
    if len(split_question) > 0:
        match_count /= len(split_question)
    question_overlap.append(match_count)
jeopardy["question_overlap"] = question_overlap

jeopardy["question_overlap"].mean()


    

0.8740103086825278

This is a test markdown cell

In [34]:
def highvalue(row):
    if row['clean_value']>800:
        value=1
    else:
        value=0
    return value 
jeopardy['high_value']=jeopardy.apply(highvalue, axis=1)

In [39]:
jeopardy.head(2)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value,answer_in_question,count_fff,question_overlap,high_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200,0.0,1,0.0,0
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe,200,0.0,1,0.0,0


In [58]:
def count_usage(term):
    high_count=0
    low_count=0
    for i,row in jeopardy.iterrows():
        split_question=row['clean_question'].split()
        if term in split_question:
            if row['high_value']==1:
                high_count=high_count+1
            else:
                low_count=low_count+1
    return (high_count,low_count)

count_usage('galileo')
observed_expected=[]
comparison_terms=list(terms_used)[0:5]

        

In [59]:
comparison_terms
for term in comparison_terms:
    observed_expected.append(count_usage(term))
    

In [65]:
observed_expected
total=0
for i in observed_expected:
    total=total+i[0]+i[1]
print(observed_expected)
print(total)

[(0, 1), (0, 1), (3, 3), (1, 0), (2, 4)]
15


In [78]:
from scipy.stats import chisquare
import numpy as np
high_value_count=len(jeopardy[jeopardy['high_value']==1])
low_value_count=len(jeopardy[jeopardy['high_value']==0])
chi_squared=[]

for i in observed_expected:
    total=i[0]+i[1]
    total_prop=total/len(jeopardy)
    expected_high_count_fiveterms=total_prop*high_value_count
    expected_low_count_fiveterms= total_prop*low_value_count
    expected= np.array([expected_high_count_fiveterms,expected_low_count_fiveterms])
    observed=np.array([i[0],i[1]])
    chi_squared.append(chisquare(observed,expected))

In [77]:
chi_squared

[Power_divergenceResult(statistic=0.39497646423335131, pvalue=0.52969509124866954),
 Power_divergenceResult(statistic=0.39497646423335131, pvalue=0.52969509124866954),
 Power_divergenceResult(statistic=1.3901593334507405, pvalue=0.23837819387149817),
 Power_divergenceResult(statistic=2.5317964247338085, pvalue=0.11157312838169751),
 Power_divergenceResult(statistic=0.074468187778142778, pvalue=0.78493885026681343)]